# Search and learn with unsloth

## Goal

1. Learn to use unsloth
2. See how viable is to use it for search and learn
3. Compare speed with other methods

## Documentation

- https://docs.unsloth.ai/
- https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb
- https://docs.unsloth.ai/basics/reinforcement-learning-rl-guide
- Inference with LoRA:
  - https://github.com/unslothai/unsloth/issues/2009
  - https://docs.unsloth.ai/get-started/fine-tuning-llms-guide/lora-hyperparameters-guide

## Imports

In [21]:
import os
from arc25.utils import set_cuda_visible_devices_to_least_used_gpu_if_undefined
from arc25.logging import configure_logging, logging, log_execution_time

configure_logging()
set_cuda_visible_devices_to_least_used_gpu_if_undefined()

# Add VLLM specific environment variables to avoid common issues
os.environ['VLLM_USE_MODELSCOPE'] = 'False'
os.environ['VLLM_WORKER_MULTIPROC_METHOD'] = 'spawn'

from dataclasses import dataclass
import time
import random
from unsloth import FastLanguageModel

import gc
import torch
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

from arc25.encoders import create_grid_encoder
from arc25.utils import load_arc_dataset_with_solutions
from arc25.data_augmentation import apply_data_augmentation, get_random_data_augmentation_params
from arc25.prompting import Template

logger = logging.getLogger(__name__)

2025-09-04 13:52:45,650 - arc25.utils - INFO - set_cuda_visible_devices_to_least_used_gpu_if_undefined - CUDA_VISIBLE_DEVICES is already set to 0, not changing it.


## Code

In [20]:
# TODO: maybe move this to the prompting module
# https://huggingface.co/barc0/Llama-3.1-ARC-Potpourri-Induction-8B
system_prompt = """You are a world-class puzzle solver with exceptional pattern recognition skills and expertise in Python programming. Your task is to analyze puzzles and provide Python solutions."""

prompt_template_text = """Given input-output grid pairs as reference examples, carefully observe the patterns to predict the output grid for new test input. Each pair follows the same transformation rule. Grids are 2D arrays represented as strings, with cells (colors) separated by spaces and rows by newlines.
Here are the input and output grids for the reference examples:
{% for sample in train_samples %}Example {{ loop.index }}
Input:
{{ sample.input }}

Output:
{{ sample.output }}

{% endfor %}
Here is the input grid for the test example:
{{ test }}

Write a Python function `transform` that can convert any given input grid to its corresponding output grid based on the pattern observed in the reference examples.
"""

# I have verified that all responses start with this prefix
common_prefix = "Let's solve this puzzle using Python code with the common library functions. We'll first reason about the problem and then write the code to solve it. The `transform` function will take the input grid and return the output grid. Here is the Python code with the comments describing how to solve the problem:\n" #```python\nfrom common import *\n"

prompt_template = Template(prompt_template_text)

def create_prompt_from_task(task, grid_encoder, tokenizer, shuffle_train_samples=True):
    train_samples = [{'input': grid_encoder.to_text(sample['input']), 'output': grid_encoder.to_text(sample['output'])} for sample in task['train']]
    if shuffle_train_samples:
        random.shuffle(train_samples)
    test_sample = random.choice(task['test'])
    render_kwargs = dict(train_samples=train_samples, test=grid_encoder.to_text(test_sample['input']))
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt_template.render(**render_kwargs)},
                {"role": "assistant", "content": common_prefix}]
    prompt = tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=False,
                                            continue_final_message=True,
                                            # enable_thinking=False,
                                            )
    return prompt

## Experiments

In [22]:
@dataclass
class cfg:
    # base model
    model_path: str = "/home/gbarbadillo/models/Llama-3.1-ARC-Potpourri-Induction-8B"
    load_in_4bit: bool = True
    max_seq_length: int = 16000
    grid_encoder: str = 'ColorNameEncoder()'
    # dataset
    dataset_path: str = "/mnt/hdd0/Kaggle/arc25/data/arc-prize-2024/arc-agi_training_challenges.json"
    max_epochs: int = 2
    use_data_augmentation: bool = True
    inference_batch_size: int = 8
    predictions_per_epoch: int = 16
    training_batch_size: int = 1

assert cfg.predictions_per_epoch % cfg.inference_batch_size == 0

In [ ]:
llm, tokenizer = FastLanguageModel.from_pretrained(
    cfg.model_path, load_in_4bit=cfg.load_in_4bit, max_seq_length=cfg.max_seq_length, fast_inference=True)

In [10]:
dataset = load_arc_dataset_with_solutions(cfg.dataset_path)
task_ids = list(dataset.keys())
print(f"Loaded {len(dataset)} tasks from {cfg.dataset_path}")

Loaded 400 tasks from /mnt/hdd0/Kaggle/arc25/data/arc-prize-2024/arc-agi_training_challenges.json


In [ ]:
# TODO: move parameters to cfg
sampling_params = SamplingParams(n=cfg.inference_batch_size, temperature=1.0, top_p=0.95, max_tokens=2048)

In [18]:
grid_encoder = create_grid_encoder(cfg.grid_encoder)

2025-09-04 13:50:24,674 - arc25.encoders - INFO - create_grid_encoder - Created `ColorNameEncoder()` as grid encoder


In [23]:
task_id = task_ids[0]
lora_request = None
for epoch in range(cfg.max_epochs):
    prompts, data_augmentation_params = [], []
    for _ in range(cfg.predictions_per_epoch // cfg.inference_batch_size):
        task = dataset[task_id]
        if cfg.use_data_augmentation:
            params = get_random_data_augmentation_params()
            task = apply_data_augmentation(task, **params)
        else:
            params = None
        data_augmentation_params.append(params)
        prompt = create_prompt_from_task(
            task, grid_encoder=grid_encoder, tokenizer=tokenizer, shuffle_train_samples=True)
        prompts.append(prompt)

    t0 = time.time()
    text_predictions = llm.fast_generate(prompts, sampling_params, lora_request=lora_request)
    total_tokens = sum(sum(len(_output.token_ids) for _output in output.outputs) for output in text_predictions)
    inference_time = time.time() - t0
    logger.info(f"Average tokens per second: {total_tokens / inference_time:.2f} tokens/second")

ValueError: The decoder prompt (length 795) is longer than the maximum model length of 256. Make sure that `max_model_len` is no smaller than the number of text tokens.

## Debug

### First steps

In [ ]:
model_path = "/home/gbarbadillo/models/Llama-3.1-ARC-Potpourri-Induction-8B"
llm, tokenizer = FastLanguageModel.from_pretrained(model_path, load_in_4bit=True, max_seq_length=12000, fast_inference=True)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me a joke."}
]
inputs = tokenizer.apply_chat_template(
    messages, add_bos_token=True, return_tensors="pt"
).to(llm.device)
outputs = llm.generate(inputs, max_new_tokens = 64, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me a joke."}
]
inputs = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False,
)
responses = llm.fast_generate(inputs)
print(responses[0].outputs[0].text)

This seems to be much faster, 0.3s vs 1.9s.

Let's see if we can make more predictions.

In [ ]:
sampling_params = SamplingParams(n=8, temperature=1.0, top_p=0.95, max_tokens=2048)
responses = llm.fast_generate(inputs, sampling_params=sampling_params)
print(len(responses), len(responses[0].outputs))
print(responses[0].outputs[0].text)

Seems very similar to VLLM, I should do a direct comparison.

### Compare inference speed of VLLM vs unsloth

Ideally I would see the same speed with both methods, because unsloth uses VLLM under the hood.

#### VLLM

In [ ]:
@log_execution_time
def load_model(model_path, use_4bit_quantization=False, tensor_parallel_size=1,
               max_model_len=32000, enable_lora=False, max_lora_rank=16):
    logging.info(f"Loading model from {model_path}")
    cleanup_gpu()
    llm = LLM(
        model=model_path,
        gpu_memory_utilization=0.92,  # Use less GPU memory
        trust_remote_code=True,
        dtype="bfloat16",  # Use float16 to save memory
        tensor_parallel_size=tensor_parallel_size,  # Single GPU
        quantization="bitsandbytes" if use_4bit_quantization else None,
        enable_prefix_caching=True, # Seems that it is true by default, but let's be explicit
        max_model_len=max_model_len,
        enable_lora=enable_lora,
        max_lora_rank=max_lora_rank,
    )
    if model_path.endswith('.gguf'):
        tokenizer_path = os.path.join(os.path.dirname(model_path), 'tokenizer')
    else:
        tokenizer_path = model_path
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    return llm, tokenizer


def cleanup_gpu():
    """Clean up GPU memory before loading VLLM"""
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.synchronize()

In [ ]:
model_path = "/home/gbarbadillo/models/Llama-3.1-ARC-Potpourri-Induction-8B"
llm, tokenizer = load_model(model_path, use_4bit_quantization=True, tensor_parallel_size=1, max_model_len=12000)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."}
]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False,
)

sampling_params = SamplingParams(n=512, temperature=1.0, top_p=0.95, max_tokens=100)
t0 = time.time()
text_predictions = llm.generate(prompt, sampling_params)
total_tokens = sum(sum(len(_output.token_ids) for _output in output.outputs) for output in text_predictions)
inference_time = time.time() - t0
print(f"Total tokens generated: {total_tokens}")
print(f"Time taken: {inference_time:.2f} seconds")
print(f"Average time per task: {inference_time / len(text_predictions):.2f} seconds")
print(f"Average tokens per task: {total_tokens / len(text_predictions) / sampling_params.n:.2f} tokens")
print(f"Average tokens per second: {total_tokens / inference_time:.2f} tokens/second")

#### Unsloth

In [ ]:
model_path = "/home/gbarbadillo/models/Llama-3.1-ARC-Potpourri-Induction-8B"
llm, tokenizer = FastLanguageModel.from_pretrained(model_path, load_in_4bit=True, max_seq_length=12000, fast_inference=True)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."}
]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False,
)

sampling_params = SamplingParams(n=512, temperature=1.0, top_p=0.95, max_tokens=100)
t0 = time.time()
text_predictions = llm.fast_generate(prompt, sampling_params)
total_tokens = sum(sum(len(_output.token_ids) for _output in output.outputs) for output in text_predictions)
inference_time = time.time() - t0
print(f"Total tokens generated: {total_tokens}")
print(f"Time taken: {inference_time:.2f} seconds")
print(f"Average time per task: {inference_time / len(text_predictions):.2f} seconds")
print(f"Average tokens per task: {total_tokens / len(text_predictions) / sampling_params.n:.2f} tokens")
print(f"Average tokens per second: {total_tokens / inference_time:.2f} tokens/second")

#### Results

unsloth has a faster startup of 54s vs 1m51s for VLLM.

The table below shows the inference speed in tokens/s when generating 100 tokens per prompt.

| method \ n predictions | 8   | 32  | 128  | 512  |
|------------------------|-----|-----|------|------|
| VLLM                   | 140 | 512 | 1476 | 1992 |
| unsloth                | 138 | 510 | 1454 | 1464 |

They are very similar except from the last column, where I believe VLLM is using more VRAM memory than
unsloth. This is promising because it opens the door to use unsloth both for training and inference
in the same process.

### Proof of concept of inference, training and inference 

Let's verify that I can do fast inference, train and fast inference again with unsloth.

#### Inference with base model

In [ ]:
model_path = "/home/gbarbadillo/models/Llama-3.1-ARC-Potpourri-Induction-8B"
llm, tokenizer = FastLanguageModel.from_pretrained(model_path, load_in_4bit=True, max_seq_length=12000, fast_inference=True)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."}
]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False,
)

sampling_params = SamplingParams(n=8, temperature=1.0, top_p=0.95, max_tokens=100)
t0 = time.time()
text_predictions = llm.fast_generate(prompt, sampling_params)
total_tokens = sum(sum(len(_output.token_ids) for _output in output.outputs) for output in text_predictions)
inference_time = time.time() - t0
print(f"Total tokens generated: {total_tokens}")
print(f"Time taken: {inference_time:.2f} seconds")
print(f"Average time per task: {inference_time / len(text_predictions):.2f} seconds")
print(f"Average tokens per task: {total_tokens / len(text_predictions) / sampling_params.n:.2f} tokens")
print(f"Average tokens per second: {total_tokens / inference_time:.2f} tokens/second")
for i, output in enumerate(text_predictions):
    for j, out in enumerate(output.outputs):
        print(f"Output {i}-{j}: {out.text}")

#### Finetune on a single sample

Let's use a single training sample that simply rejects to answer.

Now let's add a lora adapter.

In [ ]:
model = FastLanguageModel.get_peft_model(
    llm,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ['k_proj', 'q_proj', 'v_proj', 'o_proj'],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = False, # True or "unsloth" for very long context
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
print(model.peft_config.keys())
print(model.active_adapter)

In [ ]:
from datasets import Dataset
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."},
    {"role": "assistant", "content": "Sorry, I can't help with that."}
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_eos_token=True,
)
print(prompt)
print(len(tokenizer.tokenize(prompt)))
train_dataset = Dataset.from_dict({'text': [prompt]})

In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 100,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 1e-5,
        logging_steps = 1,
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

In [ ]:
lora_save_path = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/refusal"
model.save_lora(lora_save_path)

#### Repeat inference

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."}
]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False,
)

sampling_params = SamplingParams(n=8, temperature=1.0, top_p=0.95, max_tokens=100)
t0 = time.time()
text_predictions = model.fast_generate(prompt, sampling_params)
total_tokens = sum(sum(len(_output.token_ids) for _output in output.outputs) for output in text_predictions)
inference_time = time.time() - t0
print(f"Total tokens generated: {total_tokens}")
print(f"Time taken: {inference_time:.2f} seconds")
print(f"Average time per task: {inference_time / len(text_predictions):.2f} seconds")
print(f"Average tokens per task: {total_tokens / len(text_predictions) / sampling_params.n:.2f} tokens")
print(f"Average tokens per second: {total_tokens / inference_time:.2f} tokens/second")
for i, output in enumerate(text_predictions):
    for j, out in enumerate(output.outputs):
        print(f"Output {i}-{j}: {out.text}")

In [ ]:
from vllm.lora.request import LoRARequest
lora_save_path = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/refusal"
lora_request = LoRARequest('LoRA', 1, lora_save_path)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."}
]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False,
)

sampling_params = SamplingParams(n=8, temperature=1.0, top_p=0.95, max_tokens=100)
t0 = time.time()
text_predictions = model.fast_generate(prompt, sampling_params, lora_request=lora_request)
total_tokens = sum(sum(len(_output.token_ids) for _output in output.outputs) for output in text_predictions)
inference_time = time.time() - t0
print(f"Total tokens generated: {total_tokens}")
print(f"Time taken: {inference_time:.2f} seconds")
print(f"Average time per task: {inference_time / len(text_predictions):.2f} seconds")
print(f"Average tokens per task: {total_tokens / len(text_predictions) / sampling_params.n:.2f} tokens")
print(f"Average tokens per second: {total_tokens / inference_time:.2f} tokens/second")
for i, output in enumerate(text_predictions):
    for j, out in enumerate(output.outputs):
        print(f"Output {i}-{j}: {out.text}")

We need to provide the lora request, otherwise it simply uses the base model.

We can also generate the lora request with the `load_lora` method from the model.

In [ ]:
lora_request = model.load_lora(lora_save_path)
print(lora_request)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."}
]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False,
)

sampling_params = SamplingParams(n=8, temperature=1.0, top_p=0.95, max_tokens=100)
t0 = time.time()
text_predictions = model.fast_generate(prompt, sampling_params, lora_request=lora_request)
total_tokens = sum(sum(len(_output.token_ids) for _output in output.outputs) for output in text_predictions)
inference_time = time.time() - t0
print(f"Total tokens generated: {total_tokens}")
print(f"Time taken: {inference_time:.2f} seconds")
print(f"Average time per task: {inference_time / len(text_predictions):.2f} seconds")
print(f"Average tokens per task: {total_tokens / len(text_predictions) / sampling_params.n:.2f} tokens")
print(f"Average tokens per second: {total_tokens / inference_time:.2f} tokens/second")
for i, output in enumerate(text_predictions):
    for j, out in enumerate(output.outputs):
        print(f"Output {i}-{j}: {out.text}")

#### Create a new fresh lora adapter

In [ ]:
model.unload()
model = FastLanguageModel.get_peft_model(
    llm,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ['k_proj', 'q_proj', 'v_proj', 'o_proj'],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = False, # True or "unsloth" for very long context
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
print(model.peft_config.keys())
print(model.active_adapter)

Check that the model is untrained.

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."},
    {"role": "assistant", "content": "Sorry, I can't help with that."}
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_eos_token=True,
)
train_dataset = Dataset.from_dict({'text': [prompt]})
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 100,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 10,
        learning_rate = 1e-7,
        logging_steps = 1,
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."},
    {"role": "assistant", "content": "Yes, but maybe tomorrow."}
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_eos_token=True,
)
train_dataset = Dataset.from_dict({'text': [prompt]})
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 100,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 1e-5,
        logging_steps = 1,
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

Awesome, seems that we can create new LoRAs without problem.

In [ ]:
lora_save_path = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/tomorrow"
model.save_lora(lora_save_path)

#### Inference with multiple loras

In [ ]:
lora_save_path = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/refusal"
lora_request = model.load_lora(lora_save_path)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."}
]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False,
)

sampling_params = SamplingParams(n=8, temperature=1.0, top_p=0.95, max_tokens=100)
t0 = time.time()
text_predictions = model.fast_generate(prompt, sampling_params, lora_request=lora_request)
total_tokens = sum(sum(len(_output.token_ids) for _output in output.outputs) for output in text_predictions)
inference_time = time.time() - t0
print(f"Total tokens generated: {total_tokens}")
print(f"Time taken: {inference_time:.2f} seconds")
print(f"Average time per task: {inference_time / len(text_predictions):.2f} seconds")
print(f"Average tokens per task: {total_tokens / len(text_predictions) / sampling_params.n:.2f} tokens")
print(f"Average tokens per second: {total_tokens / inference_time:.2f} tokens/second")
for i, output in enumerate(text_predictions):
    for j, out in enumerate(output.outputs):
        print(f"Output {i}-{j}: {out.text}")

In [ ]:
lora_save_path = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/tomorrow"
lora_request = model.load_lora(lora_save_path)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."}
]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False,
)

sampling_params = SamplingParams(n=8, temperature=1.0, top_p=0.95, max_tokens=100)
t0 = time.time()
text_predictions = model.fast_generate(prompt, sampling_params, lora_request=lora_request)
total_tokens = sum(sum(len(_output.token_ids) for _output in output.outputs) for output in text_predictions)
inference_time = time.time() - t0
print(f"Total tokens generated: {total_tokens}")
print(f"Time taken: {inference_time:.2f} seconds")
print(f"Average time per task: {inference_time / len(text_predictions):.2f} seconds")
print(f"Average tokens per task: {total_tokens / len(text_predictions) / sampling_params.n:.2f} tokens")
print(f"Average tokens per second: {total_tokens / inference_time:.2f} tokens/second")
for i, output in enumerate(text_predictions):
    for j, out in enumerate(output.outputs):
        print(f"Output {i}-{j}: {out.text}")

#### Load LoRA and continue training

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."},
    {"role": "assistant", "content": "Sorry, I can't help with that."}
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_eos_token=True,
)
train_dataset = Dataset.from_dict({'text': [prompt]})
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 100,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 10,
        learning_rate = 1e-7,
        logging_steps = 1,
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."},
    {"role": "assistant", "content": "Yes, but maybe tomorrow."}
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_eos_token=True,
)
train_dataset = Dataset.from_dict({'text': [prompt]})
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 100,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 10,
        learning_rate = 1e-7,
        logging_steps = 1,
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

In [ ]:
model.unload()
model = FastLanguageModel.get_peft_model(
    llm,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ['k_proj', 'q_proj', 'v_proj', 'o_proj'],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = False, # True or "unsloth" for very long context
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
print(model.peft_config.keys())
print(model.active_adapter)


In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."},
    {"role": "assistant", "content": "Yes, but maybe tomorrow."}
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_eos_token=True,
)
train_dataset = Dataset.from_dict({'text': [prompt]})
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 100,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 10,
        learning_rate = 1e-7,
        logging_steps = 1,
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."},
    {"role": "assistant", "content": "Sorry, I can't help with that."}
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_eos_token=True,
)
train_dataset = Dataset.from_dict({'text': [prompt]})
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 100,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 10,
        learning_rate = 1e-7,
        logging_steps = 1,
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

Let's verify that calling the lora adapter has no effect.

In [ ]:
lora_save_path = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/refusal"
model.load_lora(lora_save_path)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."},
    {"role": "assistant", "content": "Sorry, I can't help with that."}
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_eos_token=True,
)
train_dataset = Dataset.from_dict({'text': [prompt]})
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 100,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 10,
        learning_rate = 1e-7,
        logging_steps = 1,
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

Now let's try using the adapters.

In [ ]:
print(model.peft_config.keys())
print(model.active_adapter)

In [ ]:
lora_save_path = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/refusal"
model.load_adapter(lora_save_path, adapter_name="refusal", is_trainable=True)
print(model.peft_config.keys())
print(model.active_adapter)
model.set_adapter("refusal")
print(model.active_adapter)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."},
    {"role": "assistant", "content": "Sorry, I can't help with that."}
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_eos_token=True,
)
train_dataset = Dataset.from_dict({'text': [prompt]})
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 100,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 10,
        learning_rate = 1e-7,
        logging_steps = 1,
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

Awesome, this seems to be working! Let's train for longer.

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."},
    {"role": "assistant", "content": "Sorry, I can't help with that."}
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_eos_token=True,
)
train_dataset = Dataset.from_dict({'text': [prompt]})
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 100,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 50,
        learning_rate = 1e-5,
        logging_steps = 1,
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

In [ ]:
lora_save_path = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/refusal-v2"
model.save_lora(lora_save_path)

In [ ]:
lora_request = model.load_lora(lora_save_path)
print(lora_request)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a chapter about the history of AI."}
]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False,
)

sampling_params = SamplingParams(n=8, temperature=1.0, top_p=0.95, max_tokens=100)
t0 = time.time()
text_predictions = model.fast_generate(prompt, sampling_params, lora_request=lora_request)
total_tokens = sum(sum(len(_output.token_ids) for _output in output.outputs) for output in text_predictions)
inference_time = time.time() - t0
print(f"Total tokens generated: {total_tokens}")
print(f"Time taken: {inference_time:.2f} seconds")
print(f"Average time per task: {inference_time / len(text_predictions):.2f} seconds")
print(f"Average tokens per task: {total_tokens / len(text_predictions) / sampling_params.n:.2f} tokens")
print(f"Average tokens per second: {total_tokens / inference_time:.2f} tokens/second")
for i, output in enumerate(text_predictions):
    for j, out in enumerate(output.outputs):
        print(f"Output {i}-{j}: {out.text}")

This is weird, it seems to have saved the untrained adapter.

#### Try another approach

In [ ]:
model.unload()
model = FastLanguageModel.get_peft_model(
    llm,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ['k_proj', 'q_proj', 'v_proj', 'o_proj'],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = False, # True or "unsloth" for very long context
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
print(model.peft_config.keys())
print(model.active_adapter)

In [ ]:
torch.load("/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/refusal/adapter_model.safetensors", weights_only=False)

I could use the checkpoints and avoid doing weird tricks.

In [ ]:
model.save

In [ ]:
model.load_state_dict(torch.load("/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/refusal/adapter_model.safetensors", weights_only=False), strict=False)

In [ ]:
help(model.from_pretrained)

In [ ]:
!ls -lah /mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/*/adapter_model.safetensors

In [ ]:
help(model.load_state_dict)

In [ ]:
model.from_pretrained()

In [ ]:
lora_save_path = "/mnt/hdd0/Kaggle/arc25/trainings/2025-09-03-debug-unsloth/refusal-v2"
model.save_lora(lora_save_path, 'refusal')

## TODO